In [4]:
%cd /app
# dreamaussianディレクトリがないのであれば、git cloneする
!test -d dreamgaussian || git clone -b dev https://github.com/camenduru/dreamgaussian

!pip install -q torch-ema einops tensorboardX plyfile dearpygui huggingface_hub diffusers accelerate transformers xatlas 
!pip install -q trimesh PyMCubes pymeshlab rembg[gpu,cli] omegaconf ninja gradio
!pip install -q https://github.com/camenduru/diff-gaussian-rasterization/releases/download/v1.0/diff_gaussian_rasterization-0.0.0-cp310-cp310-linux_x86_64.1.whl
!pip install -q https://github.com/camenduru/diff-gaussian-rasterization/releases/download/v1.0/simple_knn-0.0.0-cp310-cp310-linux_x86_64.1.whl
!pip install -q https://github.com/camenduru/diff-gaussian-rasterization/releases/download/v1.0/nvdiffrast-0.3.1-py3-none-any.whl
!pip install -q https://github.com/camenduru/diff-gaussian-rasterization/releases/download/v1.0/kiui-0.1.8-py3-none-any.whl
!apt install -qq -y xvfb

# 追加
!apt install -q -y python-is-python3
!pip install rich
!pip uninstall -y imageio imageio-ffmpeg
!pip install imageio==2.19.3
!pip install imageio-ffmpeg==0.4.7

%cd /app/dreamgaussian

import re
import subprocess
import gradio as gr

def create_from_text(prompt):
    sanitized_prompt = re.sub('[^0-9a-zA-Z]+', '_', prompt)
    cmd1 = ["python", "main.py", "--config", "configs/text.yaml", f"prompt={prompt}", f"save_path={sanitized_prompt}"]
    cmd2 = ["python", "main2.py", "--config", "configs/text.yaml", f"prompt={prompt}", f"save_path={sanitized_prompt}"]
    cmd3 = ["xvfb-run", "python", "-m", "kiui.render", f"logs/{sanitized_prompt}.obj", "--save_video", f"{sanitized_prompt}.mp4"]
    subprocess.run(cmd1)
    subprocess.run(cmd2)
    subprocess.run(cmd3)
    return f"{sanitized_prompt}.mp4"

def create_from_image(image):
    sanitized_prompt = 'image'
    image.save(f"{sanitized_prompt}.png")
    cmd1 = ["python", "process.py", f"{sanitized_prompt}.png", "--size", "512"] # <<< 256 may be a good default
    cmd2 = ["python", "main.py", "--config", "configs/image.yaml", f"input={sanitized_prompt}_rgba.png", f"save_path={sanitized_prompt}"]
    cmd3 = ["python", "main2.py", "--config", "configs/image.yaml", f"input={sanitized_prompt}_rgba.png", f"save_path={sanitized_prompt}"]
    cmd4 = ["xvfb-run", "python", "-m", "kiui.render", f"logs/{sanitized_prompt}.obj", "--save_video", f"{sanitized_prompt}.mp4"]
    subprocess.run(cmd1)
    subprocess.run(cmd2)
    subprocess.run(cmd3)
    subprocess.run(cmd4)
    return f"{sanitized_prompt}.mp4"

with gr.Blocks() as demo:
    prompt = gr.Textbox(lines=2, placeholder="Enter Prompt...")
    image = gr.Image(type="pil", label="Image")
    btn = gr.Button("Generate from text")
    btn_2 = gr.Button("Generate from image")
    video = gr.Video(type="mp4")
    btn.click(create_from_text, inputs=prompt, outputs=video)
    btn_2.click(create_from_image, inputs=image, outputs=video)
    
demo.queue().launch(share=True, inline=False, debug=True)

/app
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
python-is-python3 is already the newest version (3.9.2-2).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Found existing installation: imageio 2.31.4
Uninstalling imageio-2.31.4:
  Successfully uninstalled imageio-2.31.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.9 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.21.0 requires imageio>=2.27, but you have imageio 2.19.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 38.9 MB/s eta 0:00:0000:0100:01
/app/dreamgaussian


/tmp/ipykernel_52/2795663151.py:54: GradioUnusedKwargWarning: You have unused kwarg parameters in Video, please remove them: {'type': 'mp4'}
  video = gr.Video(type="mp4")


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://56b053d15e9e448ca5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


Number of points at initialisation :  1000
[INFO] loading SD...


...

[load_obj] use texture from: logs/image_mesh_albedo.png
[load_obj] load texture: (1024, 1024, 3)
[Mesh loading] v: torch.Size([32510, 3]), f: torch.Size([65080, 3])
[Mesh loading] vn: torch.Size([32510, 3]), fn: torch.Size([65080, 3])
[Mesh loading] vt: torch.Size([39394, 2]), ft: torch.Size([65080, 3])
[INFO] load image from image_rgba.png...
[INFO] loading zero123...


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 10.28it/s]


[INFO] loaded zero123!


100%|██████████| 50/50 [00:27<00:00,  1.81it/s]


[INFO] save model to logs/image.obj.
[load_obj] use texture from: logs/image_albedo.png
[load_obj] load texture: (1024, 1024, 3)
[Mesh loading] v: torch.Size([32510, 3]), f: torch.Size([65080, 3])
[Mesh loading] vn: torch.Size([32510, 3]), fn: torch.Size([65080, 3])
100%|██████████| 1/1 [00:02<00:00,  2.86s/it]
